# Particion con slacks:

Queremos ahora probar que pasa si intentamos resolver el problema de las particiones usando slacks:

$$\sum_{i = 1}^{N} a_ix_i \leq \frac{P}{2}$$
$$\sum_{i = 1}^{N} a_iy_i \leq \frac{P}{2}$$

Ademas, vamos a poner una penalizacion a $(x_i + y_i - 1)^2$ para que los conjuntos particiones $S$.
#
La matriz correspondiente a esta penalizacion $Q_{penalizacion}$ es igual a la matriz formada con identidades en el segundo metodo de particiones, con la diferencia de que vamos a tener que agregar ceros en el lugar de las variables slacks.

In [ ]:
import os
import time
import numpy as np
import pandas as pd
import math
from utils import *
import dimod
import warnings
import matplotlib.pyplot as plt
from functions_dwave import *
warnings.filterwarnings('ignore')

In [ ]:
# Creamos una instancia del problema
set = np.array([3, 1, 1, 2, 2, 1])
set = np.array([7, 15, 3, 2, 5, 5, 10, 7])
set = np.array([11, 5, 1, 5]) # Este array solo puede ser particionado de una forma
set = np.array([1, 1, 1, 1, 1, 5, 10])
#set = np.array([1, 2, 3])
set = np.array([1, 2, 3, 4])
suma = sum(set)
size = len(set)
data = dict({'set': set})
df = pd.DataFrame(data)
print("-------------------------------------")
print("Partition the set: \n ")
print(df)
print("With total sum of: ", suma)

In [ ]:
# Tomamos variables slacks para la suma total A (logA)
num_slack_total = int(math.trunc(np.log2(suma))) + 1
'''
id = np.diag(np.ones(size)) # Matriz id de size*size
id_expanded = np.diag(np.r_[np.ones(size), np.zeros(num_slack_total)])
q_disjoint = np.column_stack((-id_expanded, np.row_stack((id, np.zeros((num_slack_total, size))))))
q_disjoint = np.row_stack((q_disjoint, np.column_stack((id , np.zeros((size, num_slack_total)), -id))))
'''

# Definimos Q_disjoint igual que antes, agregando ceros para mantener las dimensiones:
id_expanded = np.diag(np.r_[np.ones(size), np.zeros(num_slack_total)])
q_disjoint = np.row_stack((np.column_stack((-id_expanded, id_expanded)), np.column_stack((id_expanded, -id_expanded))))

pesosSlacks = (2 ** (np.arange(num_slack_total - 1))).astype(float)
pesosSlacks = np.r_[pesosSlacks, suma - sum(pesosSlacks)] # revisar esto.

# Definimos Q_restriccion para cada uno de los conjuntos
vect_1 = np.r_[2 * set, pesosSlacks, np.zeros(size + num_slack_total)]
vect_2 = np.r_[np.zeros(size + num_slack_total), 2 * set, pesosSlacks]

q_restriccion_set1 = np.outer(vect_1, vect_1) - 2 * suma * np.diag(vect_1)
q_restriccion_set2 = np.outer(vect_2, vect_2) - 2 * suma * np.diag(vect_2)
q_restriccion = q_restriccion_set1 + q_restriccion_set2

alpha = 1
beta = 10
qubo = alpha * q_restriccion + beta * q_disjoint


In [ ]:
# Enviamos problema a Dwave:
print("simmulating....")
sampleset = sendToDwave(qubo, 100, True) # aggregate = True
print("Filtering:")
# Nos quedamos con la solucion que minimiza la energia:
energies = [element[1] for element in sampleset]
solution = sampleset[energies.index(min(energies))][0] # Le saco la energia
set_1 = solution[0:size]
set_2 = solution[size+num_slack_total:2*size + num_slack_total]
print(set_1)
print(set_2)
print(sampleset)


plt.bar([(str(el[1])) for el in sampleset], [(el[1]) for el in sampleset])
plt.xticks(rotation = 90)
plt.show()


In [ ]:
index_of_set_1 = np.where(set_1 == 1)[0] # retorna los indices de los items elegidos.
index_of_set_2 = np.where(set_2 == 1)[0]
output_df_1 = df.iloc[index_of_set_1]
output_df_2 = df.iloc[index_of_set_2]
print("-------------------------------------")
print("Choosen items in Set 1 are: ")
print(output_df_1)
print("-------------------------------------")
print("-------------------------------------")
print("Choosen items in Set 2 are: ")
print(output_df_2)
print("-------------------------------------")


sum_set_1 = np.dot(set, solution[0:size])
sum_set_2 = np.dot(set, solution[size+num_slack:2*size+num_slack])
print("Suma del primer conjunto = ", sum_set_1)
print("Suma del segundo conjunto = ", sum_set_2)

verificacion_disjunta = np.dot(set_1, set_2) # Si es cero, los conjuntos son efectivamente disjuntos
verificacion_particion = np.sum(set_1) + np.sum(set_2) - size # Si es cero, los conjuntos particionan S

if verificacion_disjunta != 0:
    print("ERROR: los conjuntos no son disjuntos")

if verificacion_particion != 0:
    print("ERROR: los conjuntos no particionan S")